In [ ]:
import Transform.Schedule
import nn.heads
import nn.torchvision_models


In [1]:
from Configuration import Editor

with Editor('Config') as Config:
    Config.Random.Seed = 4
    Config.Random.cuDNN.Deterministic = True
    Config.Random.cuDNN.Benchmark = True
    #prepare dataset before training
    
    Config.Dataset.ImagesRootPath = r'D:\Dataset_Collection\Cardiac_Catheterization\train\images'
    Config.Dataset.MasksRootPath = r'D:\Dataset_Collection\Cardiac_Catheterization\train\masks'
    
    Config.Dataset.IO.InputRGBImage = False
    Config.Dataset.IO.NumWorkers = 0
    Config.Dataset.IO.PinMemory = False
    Config.Dataset.IO.PrefetchFactor = 2
    Config.Dataset.IO.OutputDtype = 'float'
    
    Config.Dataset.Preprocess.Version = 'v1'
    #train dataset
    Config.Dataset.Train.BatchSize = None
    Config.Dataset.Train.Transform.Combination.Version = 'v1'
    Config.Dataset.Train.Transform.Schedule = 0.8
    Config.Dataset.Train.Transform.Combination.Components = 'default'
    Config.Dataset.Train.Transform.Combination.Params = 'default'
    Config.Dataset.Train.Transform.Combination.Schedules = 'default'
    #validation dataset
    Config.Dataset.Validation.BatchSize = None
    Config.Dataset.Validation.Ratio = 3
    Config.Dataset.Validation.Transform.Combination.Version = 'v1'
    Config.Dataset.Validation.Transform.Schedule = 0.8
    Config.Dataset.Validation.Transform.Combination.Components = 'default'
    Config.Dataset.Validation.Transform.Combination.Params = 'default'
    Config.Dataset.Validation.Transform.Combination.Schedules = 'default'
    
    
    Config.Model.Class = object
    Config.Model.Param = {}
    
    Config.Model.Load.Path = None
    Config.Model.Load.FileName = ''
    
    Config.Model.Trainable.Backbone = True
    Config.Model.Trainable.Head = True
    
    Config.Training.Loss.Class = object
    Config.Training.Loss.Param = {}
    Config.Training.Optimizer.Class = object
    Config.Training.Optimizer.HyperParam = {}
    Config.Training.Scheduler.Class = object
    Config.Training.Scheduler.HyperParam = {}
    
    Config.Training.Resume.Load.Path = None
    Config.Training.Resume.Load.Filename = str
    Config.Training.Resume.Loss = False
    Config.Training.Resume.Optimizer = False
    Config.Training.Resume.Schedule = False
    
    Config.Training.Attempts = 2
    Config.Training.Epochs = 3
    Config.Training.GradientAccumulation = 1
    
    Config.Callback.SavePath = ''
    Config.Callback.ImagesPerSave = 128
    
    Config.Logging.ImagesPerLog = 32
    Config.Logging.ProjectName = 'model_1 demo'
    Config.Logging.ModelName = 'model_1'
    Config.Logging.Comment = 'change Optimizer, Loss function'
    Config.Logging.TrialCount = 2
    
    Config.Logging.Purpose = ''
    Config.Logging.Note = ''


In [42]:
model = smp.Unet(
    encoder_name = "efficientnet-b4", 
    encoder_weights = "imagenet", 
    in_channels = 1, 
    classes = 1, 
    activation = None,
    aux_params = {'classes':5})

In [24]:
model = models.segmentation.fcn_resnet50(num_classes=10)

In [49]:
model.classification_head

ClassificationHead(
  (0): AdaptiveAvgPool2d(output_size=1)
  (1): Flatten(start_dim=1, end_dim=-1)
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=448, out_features=5, bias=True)
  (4): Activation(
    (activation): Identity()
  )
)

In [48]:
model.segmentation_head

Unet(
  (encoder): EfficientNetEncoder(
    (_conv_stem): Conv2dStaticSamePadding(
      1, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          48, 12, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          12, 48, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        

In [45]:
out = model(torch.ones((1,1,512,512),dtype=torch.float))

In [46]:
out

(tensor([[[[ 2.4951,  3.1761,  4.4296,  ...,  7.5192,  1.5934,  0.2162],
           [ 2.8385,  3.3723,  1.9058,  ...,  1.0661,  1.1474, -2.8010],
           [ 2.0627,  3.0362,  0.6236,  ...,  5.3209,  7.8046,  1.4252],
           ...,
           [-2.0838, -3.9450, -6.3793,  ...,  6.9395, 19.7731,  9.9840],
           [ 2.6996,  4.6664,  1.2459,  ..., -2.0538,  2.7597,  6.9499],
           [-2.1589, -3.2248, -1.9749,  ..., -1.4790, -4.1346,  2.7804]],
 
          [[ 1.2778,  0.8549, -1.0824,  ...,  0.0455,  0.3617, -2.0690],
           [ 1.1227,  1.4468, -0.8991,  ...,  0.7439,  4.8086, -2.7330],
           [ 2.4816,  3.5249, -2.6713,  ..., -3.8090,  0.2674, -5.7277],
           ...,
           [ 6.8531,  4.8231,  2.0487,  ..., 13.0245, 20.2685,  1.4242],
           [ 0.4553, -3.0266, -0.1472,  ..., 14.0465, 18.4490, 10.9781],
           [ 0.8012, -1.8688,  2.6358,  ..., -3.3315,  0.6053,  3.6328]],
 
          [[ 0.1193, -0.6585, -0.0564,  ...,  0.7137, -0.0681, -0.7101],
           [ 

In [ ]:
import matplotlib.pyplot as plt

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.optim import lr_scheduler
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import models

import segmentation_models_pytorch as smp

In [ ]:
import numpy as np
import os
import random
import torch

from Dataset import TrainDataLoader
import Configuration

import Transform.Preprocess
import Transform.Combinations
import Transform.Schedule

from Model import build_model


Config = Configuration.Config

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = Config.Random.Seed
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = Config.Random.cuDNN.Deterministic
torch.backends.cudnn.benchmark = Config.Random.cuDNN.Benchmark

train_transform_creater = getattr(Transform.Combinations, Config.Dataset.Train.Transform.Combination.Version)
train_transform_creater = train_transform_creater(
    Config.Dataset.Train.Transform.Schedule,
    Config.Dataset.Train.Transform.Combination.Components,
    Config.Dataset.Train.Transform.Combination.Params,
    Config.Dataset.Train.Transform.Combination.Schedules)

validation_transform_creater = getattr(Transform.Combinations, Config.Dataset.Validation.Transform.Combination.Version)
validation_transform_creater = validation_transform_creater(
    Config.Dataset.Validation.Transform.Schedule,
    Config.Dataset.Validation.Transform.Combination.Components,
    Config.Dataset.Validation.Transform.Combination.Params,
    Config.Dataset.Validation.Transform.Combination.Schedules)

preprocess = getattr(Transform.Preprocess,Config.Dataset.Preprocess.Version)

dataloader = TrainDataLoader(
    images_root = Config.Dataset.ImagesRootPath,
    masks_root = Config.Dataset.MasksRootPath,
    train_transform = train_transform_creater,
    train_batch_size = Config.Dataset.Train.BatchSize,
    validation_ratio = Config.Dataset.Validation.Ratio,
    validation_transform = validation_transform_creater,
    validation_batch_size = Config.Dataset.Validation.BatchSize,
    image_rgb = Config.Dataset.IO.InputRGBImage,
    preprocess = preprocess,
    num_workers = Config.Dataset.IO.NumWorkers,
    pin_memory = Config.Dataset.IO.PinMemory,
    prefetch_factor = Config.Dataset.IO.PrefetchFactor,
    dtype=Config.Dataset.IO.OutputDtype)

for attempt in range(Config.Training.Attempts):
    train_dataset, validation_dataset, validation_dataset_wo_arg = dataloader.get_dataset()
    print(f'===============attempt {attempt}==================')

    model = build_new_model(seed)
    
    ModelClass = Config.Model.Class

    if Config.Model.Load.Path is not None:
        load_path = pathlib.Path(Config.Model.Load.Path).joinpath(Config.Model.Load.FileName)
        load_model_state = torch.load(load_path)
        if Config.Model.Load.PretrainedModel.OverWriteHyperParams:
            Config.Model.HyperParam.update(load_model_state['hyperparam'])

    model = ModelClass(**Config.Model.HyperParam)
    if Config.Model.Load.PretrainedModel.OverWriteStateDict:
        model.load_state_dict(load_model_state['state_dict'])
    
    loss_func = Config.Training.Loss.Class(**Config.Training.Loss.Param)
    optimizer = Config.Training.Optimizer.Class(model.parameters(),**Config.Training.Optimizer.HyperParam)
    scheduler = Config.Training.Scheduler.Class(optimizer,**Config.Training.Scheduler.HyperParam)
    
    
    if Config.Training.Resume.Load.Path is not None:
        if 
    
    
    model.to(device)
    optimizer.to(device)
    scheduler.to(device)
    model.train()
    

    for epoch in range(Config.Training.Epochs):
        print(f'==========epoch {epoch}=============')

        print('=====Train=====')
        for batch_train_data in train_dataset(epoch):
            image,mask = batch_train_data
            image = image[0]
            plt.figure()
            plt.subplot(1,2,1)
            plt.imshow(image,cmap = 'gray')
            plt.axis('off')
            plt.subplot(1,2,2)
            plt.imshow(image,cmap = 'gray')
            plt.imshow(mask,alpha=0.1)
            plt.axis('off')
            plt.show()

        print('=====Validation=====')
        for batch_validation_data in validation_dataset(epoch):
            image,mask = batch_validation_data
            image = image[0]
            plt.figure()
            plt.subplot(1,2,1)
            plt.imshow(image,cmap = 'gray')
            plt.axis('off')
            plt.subplot(1,2,2)
            plt.imshow(image,cmap = 'gray')
            plt.imshow(mask,alpha=0.1)
            plt.axis('off')
            plt.show()
        
        print('=====Validation without transform=====')
        for batch_validation_data in validation_dataset_wo_arg(epoch):
            image,mask = batch_validation_data
            image = image[0]
            plt.figure()
            plt.subplot(1,2,1)
            plt.imshow(image,cmap = 'gray')
            plt.axis('off')
            plt.subplot(1,2,2)
            plt.imshow(image,cmap = 'gray')
            plt.imshow(mask,alpha=0.1)
            plt.axis('off')
            plt.show()

In [1]:
from torch import nn
from torch.nn import functional as F
from torch import optim
import torch

class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 4, 3,bias=False)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 5, 3,bias=False)
        self.fc1 = nn.Linear(5 * 3 * 3, 3,bias=False)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 5 * 3 * 3)
        x = self.fc1(x)
        return x

In [2]:
input_x = torch.ones((1,3,20,20))
y = torch.tensor([[0,0,1]],dtype=float)
loss_func = torch.nn.CrossEntropyLoss()

In [3]:
model_1 = TheModelClass()
optimizer_1 = optim.Adam(model_1.parameters())
scheduler_1 = optim.lr_scheduler.ExponentialLR(optimizer_1, gamma=0.9)

for m in model_1.parameters():
    m.data.fill_(0.1)

In [4]:
optimizer_1.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False,
   'initial_lr': 0.001,
   'params': [0, 1, 2]}]}

In [5]:
scheduler_1.state_dict()

{'gamma': 0.9,
 'base_lrs': [0.001],
 'last_epoch': 0,
 '_step_count': 1,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.001]}

In [6]:
optimizer_1.step()
scheduler_1.step()

In [7]:
optimizer_1.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.0009000000000000001,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False,
   'initial_lr': 0.001,
   'params': [0, 1, 2]}]}

In [8]:
scheduler_1.state_dict()

{'gamma': 0.9,
 'base_lrs': [0.001],
 'last_epoch': 1,
 '_step_count': 2,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.0009000000000000001]}

In [9]:
optimizer_1.step()
scheduler_1.step()

In [10]:
optimizer_1.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.0008100000000000001,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False,
   'initial_lr': 0.001,
   'params': [0, 1, 2]}]}

In [11]:
scheduler_1.state_dict()

{'gamma': 0.9,
 'base_lrs': [0.001],
 'last_epoch': 2,
 '_step_count': 3,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.0008100000000000001]}

In [12]:
optimizer_1.step()
scheduler_1.step()

In [13]:
optimizer_1.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.000729,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False,
   'initial_lr': 0.001,
   'params': [0, 1, 2]}]}

In [14]:
scheduler_1.state_dict()

{'gamma': 0.9,
 'base_lrs': [0.001],
 'last_epoch': 3,
 '_step_count': 4,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.000729]}

In [15]:
model_2 = TheModelClass()
optimizer_2 = optim.Adam(model_2.parameters())
scheduler_2 = optim.lr_scheduler.ExponentialLR(optimizer_2, gamma=0.9)

for m in model_2.parameters():
    m.data.fill_(0.1)

In [16]:
optimizer_2.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False,
   'initial_lr': 0.001,
   'params': [0, 1, 2]}]}

In [17]:
scheduler_2.state_dict()

{'gamma': 0.9,
 'base_lrs': [0.001],
 'last_epoch': 0,
 '_step_count': 1,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.001]}

In [18]:
scheduler_2.load_state_dict(scheduler_1.state_dict())

In [19]:
scheduler_2.state_dict()

{'gamma': 0.9,
 'base_lrs': [0.001],
 'last_epoch': 3,
 '_step_count': 4,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.000729]}

In [20]:
optimizer_2.step()
scheduler_2.step()

In [21]:
optimizer_2.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.0009000000000000001,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False,
   'initial_lr': 0.001,
   'params': [0, 1, 2]}]}

In [22]:
scheduler_2.state_dict()

{'gamma': 0.9,
 'base_lrs': [0.001],
 'last_epoch': 4,
 '_step_count': 5,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.0009000000000000001]}

In [56]:
model_3 = TheModelClass()
optimizer_3 = optim.Adam(model_3.parameters())
scheduler_3 = optim.lr_scheduler.ExponentialLR(optimizer_3, gamma=0.9)

for m in model_3.parameters():
    m.data.fill_(0.1)

In [57]:
optimizer_3.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False,
   'initial_lr': 0.001,
   'params': [0, 1, 2]}]}

In [58]:
scheduler_3.state_dict()

{'gamma': 0.9,
 'base_lrs': [0.001],
 'last_epoch': 0,
 '_step_count': 1,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.001]}

In [60]:
# optimizer_3.load_state_dict(optimizer_1.state_dict())
optimizer_3.param_groups[0]['lr'] = scheduler_1.get_last_lr()[0]
scheduler_3.load_state_dict(scheduler_1.state_dict())

In [61]:
optimizer_3.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.000729,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False,
   'initial_lr': 0.001,
   'params': [0, 1, 2]}]}

In [62]:
scheduler_3.state_dict()

{'gamma': 0.9,
 'base_lrs': [0.001],
 'last_epoch': 3,
 '_step_count': 4,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.000729]}

In [63]:
optimizer_3.step()
scheduler_3.step()

In [64]:
optimizer_3.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.0006561000000000001,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False,
   'initial_lr': 0.001,
   'params': [0, 1, 2]}]}

In [65]:
scheduler_3.state_dict()

{'gamma': 0.9,
 'base_lrs': [0.001],
 'last_epoch': 4,
 '_step_count': 5,
 'verbose': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.0006561000000000001]}